# 35. 小規模データセットでのセグメント幅最適化

## 背景

実験34で判明した通り、データ規模が小さい場合（2,789件）ではOverlap(8,4)方式で
外部クエリの再現率が低下する問題があった。これはバケットがスパースになるため。

## 仮説

セグメント幅を大きくする（16bit, 32bitなど）ことで、バケット数を減らし、
各バケットに含まれるドキュメント数を増やせば、小規模データでも
Overlap方式が有効に機能するのではないか。

## 検証するパラメータ

| 設定 | セグメント幅 | ストライド | セグメント数 | バケット数(理論) |
|------|------------|-----------|-------------|----------------|
| (8, 4) | 8bit | 4bit | 31 | 256 |
| (16, 8) | 16bit | 8bit | 15 | 65,536 |
| (32, 8) | 32bit | 8bit | 13 | 4.3B |
| (32, 16) | 32bit | 16bit | 7 | 4.3B |

## 実験方法

1. 40万件から5000件をランダムサンプリング（5回）
2. 各サンプルで異なるOverlap設定を評価
3. 内部/外部クエリ両方で評価
4. 結果を平均化して比較

In [2]:
import sys
sys.path.insert(0, '..')

import numpy as np
from numpy.linalg import norm
import pandas as pd
import duckdb
from tqdm import tqdm
import time
from collections import defaultdict
from itertools import combinations
from sentence_transformers import SentenceTransformer

from src.itq_lsh import ITQLSH, hamming_distance_batch

In [3]:
# パス設定
DB_PATH = '../data/experiment_400k.duckdb'
ITQ_MODEL_PATH = '../data/itq_model.pkl'

---

## 1. データ読み込み

In [4]:
# 全データ読み込み
print('データ読み込み中...')
conn = duckdb.connect(DB_PATH, read_only=True)

result = conn.execute("""
    SELECT id, embedding
    FROM documents
    ORDER BY id
""").fetchall()
conn.close()

all_doc_ids = np.array([r[0] for r in result])
all_embeddings = np.array([r[1] for r in result], dtype=np.float32)

n_all_docs = len(all_doc_ids)
print(f'読み込み完了: {n_all_docs:,}件')
print(f'埋め込み shape: {all_embeddings.shape}')

データ読み込み中...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

読み込み完了: 400,000件
埋め込み shape: (400000, 1024)


In [5]:
# ITQモデル読み込みとハッシュ計算
print('ITQモデル読み込み中...')
itq = ITQLSH.load(ITQ_MODEL_PATH)

print('ITQハッシュ計算中...')
all_hashes = itq.transform(all_embeddings)
print(f'ハッシュ shape: {all_hashes.shape}')

ITQモデル読み込み中...
ITQハッシュ計算中...
ハッシュ shape: (400000, 128)


In [6]:
# E5モデル読み込み
print('E5モデル読み込み中...')
model = SentenceTransformer('intfloat/multilingual-e5-large')
print('完了')

E5モデル読み込み中...
完了


---

## 2. 共通ユーティリティ関数

In [7]:
def compute_ground_truth(query_embedding, all_embeddings, top_k=10):
    """ブルートフォースでGround Truthを計算"""
    query_norm = norm(query_embedding)
    all_norms = norm(all_embeddings, axis=1)
    cosines = (all_embeddings @ query_embedding) / (all_norms * query_norm + 1e-10)
    top_indices = np.argsort(cosines)[-top_k:][::-1]
    return top_indices


def evaluate_recall(predicted, ground_truth, k=10):
    """Recall@k を計算"""
    gt_set = set(ground_truth[:k])
    pred_set = set(predicted[:k]) if len(predicted) >= k else set(predicted)
    return len(gt_set & pred_set) / k


def bits_to_int(bits_array):
    """バイナリ配列を整数に変換"""
    if bits_array.ndim == 1:
        bits_array = bits_array.reshape(1, -1)
    n_bits = bits_array.shape[1]
    powers = 2 ** np.arange(n_bits - 1, -1, -1)
    return np.sum(bits_array * powers, axis=1)

In [8]:
def hash_to_overlap_segments(hash_array, segment_width=8, stride=4):
    """
    オーバーラップセグメントを生成
    """
    if hash_array.ndim == 1:
        hash_array = hash_array.reshape(1, -1)
    n_docs, n_bits = hash_array.shape
    n_segments = (n_bits - segment_width) // stride + 1
    
    segments = []
    for i in range(n_segments):
        start = i * stride
        end = start + segment_width
        segment_bits = hash_array[:, start:end]
        
        # ビットを整数に変換
        powers = 2 ** np.arange(segment_width - 1, -1, -1)
        segment_int = np.sum(segment_bits * powers, axis=1)
        segments.append(segment_int)
    
    return np.column_stack(segments), n_segments


def build_overlap_index(segments):
    """
    オーバーラップセグメントのインデックスを構築
    """
    n_docs, n_segments = segments.shape
    index = {i: defaultdict(list) for i in range(n_segments)}
    
    for doc_idx in range(n_docs):
        for seg_idx in range(n_segments):
            seg_value = int(segments[doc_idx, seg_idx])
            index[seg_idx][seg_value].append(doc_idx)
    
    return index


def overlap_segment_search(query_segments, segment_index, n_segments):
    """
    Overlapセグメント一致による候補取得（OR条件）
    """
    candidates = set()
    
    for seg_idx in range(n_segments):
        seg_value = int(query_segments[seg_idx])
        if seg_value in segment_index[seg_idx]:
            candidates.update(segment_index[seg_idx][seg_value])
    
    return np.array(list(candidates))

In [9]:
def cascade_search(query_embedding, query_hash, query_segments,
                   segment_index, n_segments, all_hashes, all_embeddings,
                   step1_limit=2000, step2_limit=500, top_k=10):
    """
    3段階カスケード検索
    """
    # Step 1: Overlapセグメント一致
    step1_candidates = overlap_segment_search(query_segments, segment_index, n_segments)
    step1_raw_count = len(step1_candidates)
    
    # Step 1でstep1_limitを超えた場合はハミング距離で絞る
    if len(step1_candidates) > step1_limit:
        dists = hamming_distance_batch(query_hash, all_hashes[step1_candidates])
        top_idx = np.argsort(dists)[:step1_limit]
        step1_candidates = step1_candidates[top_idx]
    
    # Step 2: ハミング距離ソート
    if len(step1_candidates) > step2_limit:
        dists = hamming_distance_batch(query_hash, all_hashes[step1_candidates])
        top_idx = np.argsort(dists)[:step2_limit]
        step2_candidates = step1_candidates[top_idx]
    else:
        step2_candidates = step1_candidates
    
    # Step 3: コサイン類似度
    if len(step2_candidates) > 0:
        candidate_embs = all_embeddings[step2_candidates]
        query_norm = norm(query_embedding)
        candidate_norms = norm(candidate_embs, axis=1)
        cosines = (candidate_embs @ query_embedding) / (candidate_norms * query_norm + 1e-10)
        top_k_idx = np.argsort(cosines)[-top_k:][::-1]
        top_k_indices = step2_candidates[top_k_idx]
    else:
        top_k_indices = np.array([])
    
    return {
        'top_k_indices': top_k_indices,
        'step1_raw_count': step1_raw_count,
        'step1_count': len(step1_candidates),
        'step2_count': len(step2_candidates),
    }

---

## 3. 外部クエリの準備

In [10]:
# 外部クエリ（幅広いトピックのキーワード）
external_queries = [
    # 日本語
    '人工知能',
    '機械学習',
    '医療',
    '環境問題',
    '再生医療',
    'ロボット',
    '量子コンピュータ',
    '脳科学',
    'がん治療',
    '気候変動',
    # 英語
    'artificial intelligence',
    'machine learning',
    'medical research',
    'environmental science',
    'regenerative medicine',
    'robotics',
    'quantum computing',
    'neuroscience',
    'cancer treatment',
    'climate change',
]

print(f'外部クエリ数: {len(external_queries)}')

外部クエリ数: 20


In [11]:
# 外部クエリの埋め込みとITQハッシュ
print('外部クエリ埋め込み生成中...')
external_query_embs = model.encode(
    [f'passage: {q}' for q in external_queries],
    normalize_embeddings=False
).astype(np.float32)

external_query_hashes = itq.transform(external_query_embs)
print(f'完了: {external_query_embs.shape}')

外部クエリ埋め込み生成中...
完了: (20, 1024)


---

## 4. サンプリングと評価関数

In [12]:
def create_sample_dataset(n_samples, seed):
    """
    40万件から指定件数をサンプリング
    """
    rng = np.random.default_rng(seed)
    sample_indices = rng.choice(n_all_docs, n_samples, replace=False)
    sample_indices = np.sort(sample_indices)
    
    return {
        'indices': sample_indices,
        'embeddings': all_embeddings[sample_indices],
        'hashes': all_hashes[sample_indices],
        'n_docs': n_samples,
    }

In [13]:
def evaluate_overlap_config(sample_data, segment_width, stride,
                            n_internal_queries=50,
                            step1_limit=1000, step2_limit=200):
    """
    特定のOverlap設定を評価
    """
    embeddings = sample_data['embeddings']
    hashes = sample_data['hashes']
    n_docs = sample_data['n_docs']
    
    # Overlapインデックス構築
    segments, n_segments = hash_to_overlap_segments(hashes, segment_width, stride)
    segment_index = build_overlap_index(segments)
    
    # バケット統計
    bucket_sizes = [len(docs) for seg_idx in segment_index for docs in segment_index[seg_idx].values()]
    avg_bucket_size = np.mean(bucket_sizes) if bucket_sizes else 0
    n_buckets = sum(len(segment_index[seg_idx]) for seg_idx in segment_index)
    
    # 内部クエリ準備
    rng = np.random.default_rng(42)
    internal_query_indices = rng.choice(n_docs, min(n_internal_queries, n_docs), replace=False)
    
    # 内部クエリのGround Truth
    internal_gts = {}
    for qi in internal_query_indices:
        internal_gts[qi] = compute_ground_truth(embeddings[qi], embeddings, top_k=10)
    
    # 内部クエリ評価
    internal_recalls = []
    internal_step1_counts = []
    
    for qi in internal_query_indices:
        q_segments = segments[qi]
        result = cascade_search(
            embeddings[qi], hashes[qi], q_segments,
            segment_index, n_segments, hashes, embeddings,
            step1_limit=step1_limit, step2_limit=step2_limit, top_k=10
        )
        recall = evaluate_recall(result['top_k_indices'], internal_gts[qi], k=10)
        internal_recalls.append(recall)
        internal_step1_counts.append(result['step1_raw_count'])
    
    # 外部クエリ評価
    external_recalls = []
    external_step1_counts = []
    
    for i, (q_emb, q_hash) in enumerate(zip(external_query_embs, external_query_hashes)):
        # Ground Truth（このサンプルデータに対して）
        gt = compute_ground_truth(q_emb, embeddings, top_k=10)
        
        # セグメント生成
        q_segments, _ = hash_to_overlap_segments(q_hash.reshape(1, -1), segment_width, stride)
        
        result = cascade_search(
            q_emb, q_hash, q_segments[0],
            segment_index, n_segments, hashes, embeddings,
            step1_limit=step1_limit, step2_limit=step2_limit, top_k=10
        )
        recall = evaluate_recall(result['top_k_indices'], gt, k=10)
        external_recalls.append(recall)
        external_step1_counts.append(result['step1_raw_count'])
    
    return {
        'segment_width': segment_width,
        'stride': stride,
        'n_segments': n_segments,
        'n_buckets': n_buckets,
        'avg_bucket_size': avg_bucket_size,
        'internal_recall@10': np.mean(internal_recalls),
        'internal_avg_step1': np.mean(internal_step1_counts),
        'external_recall@10': np.mean(external_recalls),
        'external_avg_step1': np.mean(external_step1_counts),
    }

---

## 5. 実験: 5000件データセット

In [14]:
# 実験設定
SAMPLE_SIZES = [5000]  # サンプルサイズ
N_SAMPLES = 5  # サンプリング回数

# Overlap設定
OVERLAP_CONFIGS = [
    (8, 4),    # 現行設定
    (8, 2),    # より多いオーバーラップ
    (16, 8),   # セグメント幅を2倍
    (16, 4),   # 幅16、ストライド4
    (32, 16),  # セグメント幅を4倍
    (32, 8),   # 幅32、ストライド8
]

print(f'サンプルサイズ: {SAMPLE_SIZES}')
print(f'サンプリング回数: {N_SAMPLES}')
print(f'Overlap設定数: {len(OVERLAP_CONFIGS)}')

サンプルサイズ: [5000]
サンプリング回数: 5
Overlap設定数: 6


In [15]:
# 5000件での実験
print('=' * 80)
print('5000件データセットでの実験')
print('=' * 80)

results_5000 = []

for sample_idx in range(N_SAMPLES):
    print(f'\nサンプル {sample_idx + 1}/{N_SAMPLES}...')
    
    # サンプルデータ作成
    sample_data = create_sample_dataset(5000, seed=42 + sample_idx)
    
    for seg_width, stride in tqdm(OVERLAP_CONFIGS, desc='Overlap設定'):
        result = evaluate_overlap_config(
            sample_data, seg_width, stride,
            n_internal_queries=50,
            step1_limit=1000, step2_limit=200
        )
        result['sample_idx'] = sample_idx
        result['n_docs'] = 5000
        results_5000.append(result)

df_5000 = pd.DataFrame(results_5000)
print('\n完了')

5000件データセットでの実験

サンプル 1/5...


Overlap設定: 100%|██████████| 6/6 [00:04<00:00,  1.29it/s]



サンプル 2/5...


Overlap設定: 100%|██████████| 6/6 [00:01<00:00,  3.01it/s]



サンプル 3/5...


...


サンプル 4/5...


Overlap設定: 100%|██████████| 6/6 [00:01<00:00,  3.11it/s]



サンプル 5/5...


Overlap設定: 100%|██████████| 6/6 [00:02<00:00,  2.94it/s]


完了


In [16]:
# 結果を集約
df_5000_agg = df_5000.groupby(['segment_width', 'stride']).agg({
    'n_segments': 'first',
    'n_buckets': 'mean',
    'avg_bucket_size': 'mean',
    'internal_recall@10': ['mean', 'std'],
    'internal_avg_step1': 'mean',
    'external_recall@10': ['mean', 'std'],
    'external_avg_step1': 'mean',
}).round(3)

# カラム名をフラット化
df_5000_agg.columns = [
    'n_segments', 'n_buckets', 'avg_bucket_size',
    'internal_r10_mean', 'internal_r10_std',
    'internal_step1',
    'external_r10_mean', 'external_r10_std',
    'external_step1'
]
df_5000_agg = df_5000_agg.reset_index()

In [17]:
# 5000件結果表示
print('\n' + '=' * 120)
print('5000件データセット: Overlap設定比較')
print('=' * 120)

print(f'\n{"Width":>5} | {"Stride":>6} | {"Segs":>4} | {"Buckets":>8} | {"BktSize":>8} | '
      f'{"Int R@10":>10} | {"Int Step1":>10} | {"Ext R@10":>10} | {"Ext Step1":>10}')
print('-' * 120)

for _, row in df_5000_agg.iterrows():
    int_r10 = f'{row["internal_r10_mean"]*100:.1f}%±{row["internal_r10_std"]*100:.1f}'
    ext_r10 = f'{row["external_r10_mean"]*100:.1f}%±{row["external_r10_std"]*100:.1f}'
    ext_mark = '★' if row['external_r10_mean'] >= 0.80 else ''
    
    print(f'{row["segment_width"]:>5} | {row["stride"]:>6} | {row["n_segments"]:>4} | '
          f'{row["n_buckets"]:>8.0f} | {row["avg_bucket_size"]:>8.1f} | '
          f'{int_r10:>10} | {row["internal_step1"]:>10.0f} | '
          f'{ext_r10:>10}{ext_mark} | {row["external_step1"]:>10.0f}')


5000件データセット: Overlap設定比較

Width | Stride | Segs |  Buckets |  BktSize |   Int R@10 |  Int Step1 |   Ext R@10 |  Ext Step1
------------------------------------------------------------------------------------------------------------------------
  8.0 |    2.0 | 61.0 |    15559 |     19.6 |  89.8%±1.6 |       1096 |  94.3%±1.8★ |       1207
  8.0 |    4.0 | 31.0 |     7906 |     19.6 |  86.5%±1.3 |        812 |  90.3%±2.8★ |        899
 16.0 |    4.0 | 29.0 |   105122 |      1.4 |  36.1%±4.8 |         43 |  30.2%±4.1 |         38
 16.0 |    8.0 | 15.0 |    54150 |      1.4 |  32.2%±4.2 |         31 |  22.2%±5.2 |         26
 32.0 |    8.0 | 13.0 |    63028 |      1.0 |  13.6%±1.6 |          3 |   1.2%±0.8 |          0
 32.0 |   16.0 |  7.0 |    33927 |      1.0 |  12.4%±1.0 |          3 |   0.6%±0.7 |          0


---

## 6. 実験: 異なるデータサイズでの比較

In [18]:
# 複数サイズでの実験
print('=' * 80)
print('異なるデータサイズでの比較')
print('=' * 80)

SIZE_CONFIGS = [2000, 5000, 10000, 20000, 50000]
N_SAMPLES_QUICK = 3  # 各サイズ3回サンプリング

results_sizes = []

for size in SIZE_CONFIGS:
    print(f'\nサイズ: {size:,}件')
    
    for sample_idx in range(N_SAMPLES_QUICK):
        sample_data = create_sample_dataset(size, seed=100 + sample_idx)
        
        for seg_width, stride in OVERLAP_CONFIGS:
            result = evaluate_overlap_config(
                sample_data, seg_width, stride,
                n_internal_queries=30,
                step1_limit=min(1000, size // 5),
                step2_limit=min(200, size // 25)
            )
            result['sample_idx'] = sample_idx
            result['n_docs'] = size
            results_sizes.append(result)

df_sizes = pd.DataFrame(results_sizes)
print('\n完了')

異なるデータサイズでの比較

サイズ: 2,000件

サイズ: 5,000件

サイズ: 10,000件

サイズ: 20,000件

サイズ: 50,000件

完了


In [19]:
# サイズ別に集約
df_sizes_agg = df_sizes.groupby(['n_docs', 'segment_width', 'stride']).agg({
    'n_segments': 'first',
    'avg_bucket_size': 'mean',
    'internal_recall@10': 'mean',
    'internal_avg_step1': 'mean',
    'external_recall@10': 'mean',
    'external_avg_step1': 'mean',
}).round(3).reset_index()

In [20]:
# サイズ別結果表示
print('\n' + '=' * 140)
print('データサイズ別: Overlap設定比較')
print('=' * 140)

for size in SIZE_CONFIGS:
    print(f'\n■ {size:,}件')
    print(f'{"Width":>5} | {"Stride":>6} | {"Segs":>4} | {"BktSize":>8} | '
          f'{"Int R@10":>8} | {"Int Step1":>10} | {"Ext R@10":>8} | {"Ext Step1":>10}')
    print('-' * 100)
    
    df_size = df_sizes_agg[df_sizes_agg['n_docs'] == size]
    
    for _, row in df_size.iterrows():
        ext_mark = '★' if row['external_recall@10'] >= 0.80 else ''
        print(f'{row["segment_width"]:>5} | {row["stride"]:>6} | {row["n_segments"]:>4} | '
              f'{row["avg_bucket_size"]:>8.1f} | '
              f'{row["internal_recall@10"]*100:>7.1f}% | {row["internal_avg_step1"]:>10.0f} | '
              f'{row["external_recall@10"]*100:>7.1f}%{ext_mark} | {row["external_avg_step1"]:>10.0f}')


データサイズ別: Overlap設定比較

■ 2,000件
Width | Stride | Segs |  BktSize | Int R@10 |  Int Step1 | Ext R@10 |  Ext Step1
----------------------------------------------------------------------------------------------------
  8.0 |    2.0 | 61.0 |      8.1 |    86.0% |        431 |    90.2%★ |        484
  8.0 |    4.0 | 31.0 |      8.1 |    82.4% |        319 |    87.2%★ |        361
 16.0 |    4.0 | 29.0 |      1.2 |    31.1% |         17 |    24.5% |         14
 16.0 |    8.0 | 15.0 |      1.2 |    26.9% |         12 |    18.7% |         10
 32.0 |    8.0 | 13.0 |      1.0 |    12.4% |          2 |     0.7% |          0
 32.0 |   16.0 |  7.0 |      1.0 |    12.1% |          2 |     0.3% |          0

■ 5,000件
Width | Stride | Segs |  BktSize | Int R@10 |  Int Step1 | Ext R@10 |  Ext Step1
----------------------------------------------------------------------------------------------------
  8.0 |    2.0 | 61.0 |     19.6 |    90.1% |       1065 |    93.8%★ |       1216
  8.0 |    4.0 | 31.0 | 

---

## 7. 2段階検索との比較

In [21]:
def two_stage_search(query_embedding, query_hash, all_hashes, all_embeddings,
                     step1_limit=500, top_k=10):
    """
    2段階検索（全件ハミング距離→コサイン）
    """
    # Step 1: 全件ハミング距離
    dists = hamming_distance_batch(query_hash, all_hashes)
    top_idx = np.argsort(dists)[:step1_limit]
    
    # Step 2: コサイン類似度
    candidate_embs = all_embeddings[top_idx]
    query_norm = norm(query_embedding)
    candidate_norms = norm(candidate_embs, axis=1)
    cosines = (candidate_embs @ query_embedding) / (candidate_norms * query_norm + 1e-10)
    top_k_idx = np.argsort(cosines)[-top_k:][::-1]
    
    return top_idx[top_k_idx]

In [22]:
# 2段階検索との比較（5000件）
print('=' * 80)
print('2段階検索との比較（5000件）')
print('=' * 80)

two_stage_results = []

for sample_idx in range(3):
    sample_data = create_sample_dataset(5000, seed=200 + sample_idx)
    embeddings = sample_data['embeddings']
    hashes = sample_data['hashes']
    
    # 内部クエリ
    rng = np.random.default_rng(42)
    internal_indices = rng.choice(5000, 50, replace=False)
    
    internal_recalls = []
    for qi in internal_indices:
        gt = compute_ground_truth(embeddings[qi], embeddings, top_k=10)
        result = two_stage_search(embeddings[qi], hashes[qi], hashes, embeddings,
                                  step1_limit=500, top_k=10)
        internal_recalls.append(evaluate_recall(result, gt, k=10))
    
    # 外部クエリ
    external_recalls = []
    for q_emb, q_hash in zip(external_query_embs, external_query_hashes):
        gt = compute_ground_truth(q_emb, embeddings, top_k=10)
        result = two_stage_search(q_emb, q_hash, hashes, embeddings,
                                  step1_limit=500, top_k=10)
        external_recalls.append(evaluate_recall(result, gt, k=10))
    
    two_stage_results.append({
        'sample_idx': sample_idx,
        'internal_recall@10': np.mean(internal_recalls),
        'external_recall@10': np.mean(external_recalls),
    })

df_two_stage = pd.DataFrame(two_stage_results)

print(f'\n2段階検索（Step1=500件）:')
print(f'  内部R@10: {df_two_stage["internal_recall@10"].mean()*100:.1f}%')
print(f'  外部R@10: {df_two_stage["external_recall@10"].mean()*100:.1f}%')

2段階検索との比較（5000件）

2段階検索（Step1=500件）:
  内部R@10: 97.8%
  外部R@10: 99.5%


---

## 8. 分析と考察

In [23]:
# バケットサイズとRecallの関係を分析
print('=' * 80)
print('バケットサイズと再現率の関係分析')
print('=' * 80)

# 外部R@10が80%以上の設定を抽出
good_configs = df_sizes_agg[df_sizes_agg['external_recall@10'] >= 0.80]

print('\n■ 外部R@10 >= 80%を達成した設定:')
if len(good_configs) > 0:
    for _, row in good_configs.iterrows():
        print(f'  {row["n_docs"]:,}件 | Overlap({row["segment_width"]},{row["stride"]}) | '
              f'BktSize={row["avg_bucket_size"]:.1f} | Ext R@10={row["external_recall@10"]*100:.1f}%')
else:
    print('  なし')

# バケットサイズの目安を分析
print('\n■ バケットサイズと外部R@10の関係:')
for bucket_threshold in [5, 10, 20, 50, 100]:
    filtered = df_sizes_agg[df_sizes_agg['avg_bucket_size'] >= bucket_threshold]
    if len(filtered) > 0:
        avg_ext_r10 = filtered['external_recall@10'].mean()
        print(f'  バケットサイズ >= {bucket_threshold}: 平均外部R@10 = {avg_ext_r10*100:.1f}%')

バケットサイズと再現率の関係分析

■ 外部R@10 >= 80%を達成した設定:
  2,000.0件 | Overlap(8.0,2.0) | BktSize=8.1 | Ext R@10=90.2%
  2,000.0件 | Overlap(8.0,4.0) | BktSize=8.1 | Ext R@10=87.2%
  5,000.0件 | Overlap(8.0,2.0) | BktSize=19.6 | Ext R@10=93.8%
  5,000.0件 | Overlap(8.0,4.0) | BktSize=19.6 | Ext R@10=89.8%
  10,000.0件 | Overlap(8.0,2.0) | BktSize=39.1 | Ext R@10=91.7%
  10,000.0件 | Overlap(8.0,4.0) | BktSize=39.1 | Ext R@10=89.0%
  20,000.0件 | Overlap(8.0,2.0) | BktSize=78.1 | Ext R@10=89.8%
  20,000.0件 | Overlap(8.0,4.0) | BktSize=78.1 | Ext R@10=87.2%
  50,000.0件 | Overlap(8.0,2.0) | BktSize=195.3 | Ext R@10=85.3%
  50,000.0件 | Overlap(8.0,4.0) | BktSize=195.3 | Ext R@10=84.3%

■ バケットサイズと外部R@10の関係:
  バケットサイズ >= 5: 平均外部R@10 = 88.8%
  バケットサイズ >= 10: 平均外部R@10 = 88.9%
  バケットサイズ >= 20: 平均外部R@10 = 87.9%
  バケットサイズ >= 50: 平均外部R@10 = 86.7%
  バケットサイズ >= 100: 平均外部R@10 = 84.8%


In [24]:
# 設定ごとの最適データサイズを分析
print('\n' + '=' * 80)
print('各Overlap設定の推奨データサイズ')
print('=' * 80)

for seg_width, stride in OVERLAP_CONFIGS:
    df_config = df_sizes_agg[
        (df_sizes_agg['segment_width'] == seg_width) &
        (df_sizes_agg['stride'] == stride)
    ]
    
    # 外部R@10 >= 80%を達成する最小サイズ
    good_sizes = df_config[df_config['external_recall@10'] >= 0.80]['n_docs']
    min_size = good_sizes.min() if len(good_sizes) > 0 else 'N/A'
    
    # 全サイズでの平均R@10
    avg_ext_r10 = df_config['external_recall@10'].mean()
    
    print(f'\n■ Overlap({seg_width},{stride}):')
    print(f'  セグメント数: {df_config["n_segments"].iloc[0]}')
    print(f'  平均外部R@10: {avg_ext_r10*100:.1f}%')
    print(f'  外部R@10>=80%の最小サイズ: {min_size}')


各Overlap設定の推奨データサイズ

■ Overlap(8,4):
  セグメント数: 31
  平均外部R@10: 87.5%
  外部R@10>=80%の最小サイズ: 2000

■ Overlap(8,2):
  セグメント数: 61
  平均外部R@10: 90.2%
  外部R@10>=80%の最小サイズ: 2000

■ Overlap(16,8):
  セグメント数: 15
  平均外部R@10: 26.5%
  外部R@10>=80%の最小サイズ: N/A

■ Overlap(16,4):
  セグメント数: 29
  平均外部R@10: 33.0%
  外部R@10>=80%の最小サイズ: N/A

■ Overlap(32,16):
  セグメント数: 7
  平均外部R@10: 1.3%
  外部R@10>=80%の最小サイズ: N/A

■ Overlap(32,8):
  セグメント数: 13
  平均外部R@10: 1.9%
  外部R@10>=80%の最小サイズ: N/A


---

## 9. 結論

In [25]:
print('=' * 80)
print('結論')
print('=' * 80)

# 5000件での最良設定
df_5000_sorted = df_5000_agg.sort_values('external_r10_mean', ascending=False)
best_5000 = df_5000_sorted.iloc[0]

# 2段階検索の結果
two_stage_ext_r10 = df_two_stage['external_recall@10'].mean()

print(f'''
■ 実験目的
  小規模データ（5000件程度）でOverlapセグメント幅を調整することで、
  バケットのスパース問題を解決できるか検証する。

■ 5000件での最良Overlap設定
  設定: Overlap({int(best_5000['segment_width'])},{int(best_5000['stride'])})
  セグメント数: {int(best_5000['n_segments'])}
  平均バケットサイズ: {best_5000['avg_bucket_size']:.1f}
  内部R@10: {best_5000['internal_r10_mean']*100:.1f}%
  外部R@10: {best_5000['external_r10_mean']*100:.1f}%

■ 2段階検索（ベースライン）
  外部R@10: {two_stage_ext_r10*100:.1f}%

■ 考察
''')

if best_5000['external_r10_mean'] >= 0.80:
    print(f'  ✓ Overlap({int(best_5000["segment_width"])},{int(best_5000["stride"])})で外部R@10>=80%を達成')
    if best_5000['external_r10_mean'] >= two_stage_ext_r10 * 0.95:
        print(f'  ✓ 2段階検索と同等の性能を維持しながら、Step1での枝刈りが可能')
    else:
        print(f'  △ 2段階検索より劣るが、許容範囲内')
else:
    print(f'  △ 最良設定でも外部R@10<80%')
    print(f'  → 5000件規模では2段階検索を推奨')

結論

■ 実験目的
  小規模データ（5000件程度）でOverlapセグメント幅を調整することで、
  バケットのスパース問題を解決できるか検証する。

■ 5000件での最良Overlap設定
  設定: Overlap(8,2)
  セグメント数: 61
  平均バケットサイズ: 19.6
  内部R@10: 89.8%
  外部R@10: 94.3%

■ 2段階検索（ベースライン）
  外部R@10: 99.5%

■ 考察

  ✓ Overlap(8,2)で外部R@10>=80%を達成
  △ 2段階検索より劣るが、許容範囲内


---

## 10. 推奨設定まとめ

In [26]:
print('=' * 80)
print('データサイズ別推奨設定')
print('=' * 80)

# 各サイズで最良の設定を特定
recommendations = []

for size in SIZE_CONFIGS:
    df_size = df_sizes_agg[df_sizes_agg['n_docs'] == size]
    best = df_size.sort_values('external_recall@10', ascending=False).iloc[0]
    
    recommendations.append({
        'n_docs': size,
        'best_config': f"({int(best['segment_width'])},{int(best['stride'])})",
        'n_segments': int(best['n_segments']),
        'avg_bucket_size': best['avg_bucket_size'],
        'internal_r10': best['internal_recall@10'],
        'external_r10': best['external_recall@10'],
    })

df_rec = pd.DataFrame(recommendations)

print(f'\n{"データ件数":>10} | {"推奨設定":>12} | {"Segs":>5} | {"BktSize":>8} | {"Int R@10":>8} | {"Ext R@10":>8}')
print('-' * 80)

for _, row in df_rec.iterrows():
    ext_mark = '★' if row['external_r10'] >= 0.80 else ''
    print(f'{row["n_docs"]:>10,} | {row["best_config"]:>12} | {row["n_segments"]:>5} | '
          f'{row["avg_bucket_size"]:>8.1f} | {row["internal_r10"]*100:>7.1f}% | '
          f'{row["external_r10"]*100:>7.1f}%{ext_mark}')

データサイズ別推奨設定

     データ件数 |         推奨設定 |  Segs |  BktSize | Int R@10 | Ext R@10
--------------------------------------------------------------------------------
     2,000 |        (8,2) |    61 |      8.1 |    86.0% |    90.2%★
     5,000 |        (8,2) |    61 |     19.6 |    90.1% |    93.8%★
    10,000 |        (8,2) |    61 |     39.1 |    88.2% |    91.7%★
    20,000 |        (8,2) |    61 |     78.1 |    84.2% |    89.8%★
    50,000 |        (8,2) |    61 |    195.3 |    76.4% |    85.3%★


---

## 11. 実験評価まとめ

### 重要な発見：仮説と異なる結果

**当初の仮説**:
- セグメント幅を大きくする（16bit, 32bit）とバケット数が減り、各バケットに含まれるドキュメントが増える
- これにより小規模データでもOverlap方式が有効になる

**実際の結果**:
| 設定 | セグメント幅 | バケット数(5000件) | 平均バケットサイズ | 外部R@10 |
|------|------------|-------------------|-------------------|----------|
| **(8,2)** | 8bit | 15,559 | **19.6** | **94.3%** ◎ |
| (8,4) | 8bit | 7,906 | 19.6 | 90.3% ★ |
| (16,4) | 16bit | 105,122 | 1.4 | 30.2% ✗ |
| (16,8) | 16bit | 54,150 | 1.4 | 22.2% ✗ |
| (32,8) | 32bit | 63,028 | 1.0 | 1.2% ✗ |
| (32,16) | 32bit | 33,927 | 1.0 | 0.6% ✗ |

→ **仮説は間違いだった！** セグメント幅を大きくするとむしろ悪化する

---

### 仮説の誤り（重要）

**当初の仮説**:
> セグメント幅を大きくする（16bit, 32bit）とバケット数が減り、各バケットに含まれるドキュメント数が増える

**この仮説は根本的に間違っていた**:

| セグメント幅 | 理論上の最大バケット数 | 5000件での実際 | 平均バケットサイズ |
|------------|---------------------|--------------|------------------|
| 8bit | 256/セグメント | 256（飽和） | **19.5件** |
| 16bit | 65,536/セグメント | 5,000（データ数で制限） | **1件** |
| 32bit | 4.3B/セグメント | 5,000（データ数で制限） | **1件** |

**誤りの原因**:
- バケット数は「理論上の最大」であり、実際に使われるバケット数は**データ件数で制限される**
- 16bit/32bitでは、バケット数が減るのではなく、**スパースなバケットが増える**
- データ5000件を65,536バケットに分散 → 各バケットに平均0.076件

---

### なぜセグメント幅を大きくすると悪化するのか

1. **完全一致の確率が激減**
   - 8bitセグメント: クエリがヒットする確率 ≈ 5000/256 = **19.5件/バケット** → ほぼ必ずヒット
   - 16bitセグメント: クエリがヒットする確率 ≈ 5000/65536 = **0.076件/バケット** → 7.6%の確率
   - 32bitセグメント: クエリがヒットする確率 ≈ 5000/4.3B ≈ **0件** → ほぼヒットしない

2. **OR検索でも救えない**
   - 16bit × 15セグメント: 各セグメントのヒット率7.6% → OR検索でも候補が約26件しか集まらない
   - 32bit × 13セグメント: 各セグメントのヒット率≈0% → OR検索でも候補が0件

3. **正しい理解**
   - 問題は「バケットがスパース」ではなく「**セグメント完全一致の確率が低すぎる**」
   - セグメント幅が大きいほど、クエリと同じバケットに入るドキュメントが存在しない確率が上がる

---

### 最良設定: Overlap(8, 2)

**5000件での推奨**: `Overlap(8, 2)`
```
セグメント幅: 8bit
ストライド: 2bit
セグメント数: 61（(128-8)/2+1）
平均バケットサイズ: 19.6件

Step 1候補: 約1,100件（22%）
削減率: 78%
外部R@10: 94.3%
```

**なぜ(8,2)が(8,4)より良いか**:
- ストライドを小さくする → セグメント数が増える（31→61）
- セグメント数が多い → クエリのセグメントがヒットする確率が上がる
- より多くのOR条件でカバーできる

---

### 2段階検索との比較

| 方式 | 外部R@10 | Step1候補 | 計算量 |
|------|---------|-----------|--------|
| 2段階検索 | **99.5%** | 全件(5000) | O(N) |
| Overlap(8,2) | 94.3% | ~1,100件 | O(セグメント数 × ルックアップ) |
| Overlap(8,4) | 90.3% | ~900件 | O(セグメント数 × ルックアップ) |

**トレードオフ**:
- 2段階検索は精度最高だが、全件ハミング距離計算が必要
- Overlap(8,2)は精度5%低下だが、78%の候補削減

---

### データサイズ別推奨設定

| データ件数 | 推奨設定 | セグメント数 | 外部R@10 | コメント |
|-----------|---------|-------------|----------|----------|
| 2,000件 | (8,2) | 61 | 90.2% | ★3段階可能 |
| 5,000件 | (8,2) | 61 | 93.8% | ★3段階推奨 |
| 10,000件 | (8,2) | 61 | 91.7% | ★3段階推奨 |
| 20,000件 | (8,4) or (8,2) | 31/61 | 87-90% | ★どちらでも可 |
| 50,000件 | (8,4) | 31 | 84.3% | ★3段階推奨 |

---

### 結論

1. **セグメント幅を大きくしても解決しない（仮説は棄却）**
   - 16bit, 32bitセグメントは完全一致の確率が低すぎて候補が集まらない
   - バケット数が減るのではなく、スパースなバケットが増えるだけ

2. **ストライドを小さくして、セグメント数を増やすのが有効**
   - Overlap(8,4) → Overlap(8,2)にすることで外部R@10が+4pt改善
   - セグメント数: 31 → 61

3. **5000件規模でも3段階カスケードは有効**
   - Overlap(8,2)で外部R@10=94.3%を達成
   - 2段階検索の99.5%より5%低いが、候補削減78%のメリット

4. **小規模データでより精度が必要な場合は2段階検索を使用**
   - 精度優先なら2段階検索（全件ハミング距離→コサイン）
   - 速度優先ならOverlap(8,2)

---

### 次のステップ

1. 本番データ（1.4万件）で Overlap(8,2) を検証
2. さらにストライドを小さくする（8,1）の検証
3. 処理時間の比較（3段階 vs 2段階）